# Retreival Augemented Generation

A few steps required to make this work:

- Create some callable functions that can query the database for related keywords
- Give these callable functions to an OpenAI assistant.
- Build in recursive functions that allow the bot to query the database multiple times to pull together an outfit.
- Create a system whereby the user comminicates with a chatbot which can have long form discussions.


In [1]:
import numpy as np
import hnswlib
import clip
import torch
import os
import sys
import psutil
from PIL import Image
from IPython.display import display
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
GPT_API_KEY = os.getenv('PERSONAL_OPENAI_KEY')

from openai import OpenAI
import json

client = OpenAI(api_key=GPT_API_KEY)

# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Load HNSWlib index and image ids
hnsw_index_path = '../data/processed/hnsw_index.bin'
image_ids_path = '../data/processed/image_ids.npy'
image_folder_path = '../data/raw/images'

# Load the index
hnsw_index = hnswlib.Index(space='l2', dim=512)
hnsw_index.load_index(hnsw_index_path)
image_ids = np.load(image_ids_path)

sys.path.append('C:/projects/python/GPT-Image-Reccomender')

from src.vector_search import generate_text_embedding, search_similar_images, display_images, monitor_resources, query_images


In [2]:
def define_tools():
    tools = []

    tools.append({
        "type": "function",
        "function": {
            "name": "query_database",
            "description": "This is a database that contains images of +50000 items of clothing. This function allows you to send an unstructured text query to the database to get a list of appropriate items. Every query should include colour, style, size, gender, and item",
            "parameters": {
                "type": "object",
                "properties": {
                    "item_name": {
                        "type": "string",
                        "description": "The name of the clothing item (e.g., 'brown sandal', 'white top')."
                    },
                    "query": {
                        "type": "string",
                        "description": "Unstructured text query describing the clothing items (e.g., 'brown sandal and white top')."
                    }
                },
                "required": ["item_name", "query"]
            },
        }
    })

    return tools

def handle_function_call(function_name, function_args, top_k):
    if function_name == "query_database":
        item_name = function_args['item_name']
        query = function_args['query']
        print(f"Querying database for: {query}")
        return query_images(query, top_k=top_k)
    return {}

In [3]:
def run_conversation(user_query, introduction, top_k, max_depth, session_messages=None):
    print(max_depth)
    if session_messages is None:
        session_messages = []

    if not session_messages:
        session_messages.append({"role": "system", "content": introduction})

    tools = define_tools()

    all_image_paths_dict = {}
    all_responses = []

    session_messages.append({"role": "user", "content": user_query})
    depth = 0

    while depth < max_depth:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=session_messages,
            tools=tools,
            tool_choice="auto"
        )

        response_content = response.choices[0].message.content

        print(response)
        
        if response_content:
            all_responses.append(response_content)
            session_messages.append({"role": "assistant", "content": response_content})

        finish_reason = response.choices[0].finish_reason
        if finish_reason == "stop":
            break
        elif finish_reason == "tool_calls":
            tool_calls = response.choices[0].message.tool_calls
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}
                print('functions')
                print(function_name)
                print(function_args)
                function_response = handle_function_call(function_name, function_args, top_k)
                session_messages.append({"role": "system", "name": function_name, "content": json.dumps(function_response)})

                if function_response:
                    session_messages.append({"role": "system", "name": function_name, "content": json.dumps(function_response)})

                    if function_name == "query_database":
                        for item_name, paths in function_response.items():
                            if item_name not in all_image_paths_dict:
                                all_image_paths_dict[item_name] = []
                            all_image_paths_dict[item_name].extend(paths)
            
                
            depth += 1
        else:
            print(f'Unhandled finish reason: {finish_reason}')
            break

    return all_responses, session_messages, all_image_paths_dict


In [13]:
# Example usage
top_k = 5
user_query = "I am a girl going to a festival, could you please suggest an outfit I can wear!"
introduction = """You are a shopping assistant giving suggestions to a customer. 
You can only make one call to the database per item, if the if you suggest three different items of clothing you must make three separate calls.
If someone asks for a full outfit suggestion, please suggest everything from shirt, shoes, trousers, skirt, shorts, glasses. Whatever you think works best.
If the user asked for suggestions you must query the database with options you think would work. 
Be specific and then tell the user why you made that decision.
Please return all the images that have been returned by the database. They will be shown in markdown, so show them all side by side together.
You can make multiple function calls at one.
|  |  |  |  |  |
|:---:|:---:|:---:|:---:|:---:|
|![](../data/raw/images\\12170.jpg)|![](../data/raw/images\\12471.jpg)|![](../data/raw/images\\25529.jpg)|![](../data/raw/images\\19189.jpg)|![](../data/raw/images\\38086.jpg)|

"""

responses, session_messages, all_image_paths_dict = run_conversation(user_query, introduction, top_k, max_depth=10)

10
ChatCompletion(id='chatcmpl-9cWN2q8ihaLDvBpHX76YWR7gyU6AW', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure! Festivals are a great place to showcase fun, vibrant, and trendy outfits. I'll search for each item needed:\n\n1. A stylish top\n2. Comfortable yet fashionable bottom (shorts or skirt)\n3. Suitable shoes for walking\n4. Cool accessories like sunglasses or a hat\n\nLet me query the database to find some great suggestions for you.", role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_GquVedJSfMpgKe3iknC7YD1R', function=Function(arguments='{"item_name": "festival top", "query": "colorful, trendy, size M/L, girl\'s top for a festival"}', name='query_database'), type='function'), ChatCompletionMessageToolCall(id='call_HlTopEOwCI6t90Smtrb0QBQF', function=Function(arguments='{"item_name": "festival bottom", "query": "comfortable, stylish, size M/L, girl\'s shorts or skirt for a festi

In [14]:
print(session_messages) 

[{'role': 'system', 'content': 'You are a shopping assistant giving suggestions to a customer. \nYou can only make one call to the database per item, if the if you suggest three different items of clothing you must make three separate calls.\nIf someone asks for a full outfit suggestion, please suggest everything from shirt, shoes, trousers, skirt, shorts, glasses. Whatever you think works best.\nIf the user asked for suggestions you must query the database with options you think would work. \nBe specific and then tell the user why you made that decision.\nPlease return all the images that have been returned by the database. They will be shown in markdown, so show them all side by side together.\nYou can make multiple function calls at one.\n|  |  |  |  |  |\n|:---:|:---:|:---:|:---:|:---:|\n|![](../data/raw/images\\12170.jpg)|![](../data/raw/images\\12471.jpg)|![](../data/raw/images\\25529.jpg)|![](../data/raw/images\\19189.jpg)|![](../data/raw/images\\38086.jpg)|\n\n'}, {'role': 'use

In [15]:
from IPython.display import display, Markdown

def generate_markdown_from_session(session_messages):
    markdown_content = ""
    for message in session_messages:
        role = message.get('role')
        content = message.get('content')
        name = message.get('name', '')

        if role == 'system':
            # if 'query_database' in name:
            #     content_dict = json.loads(content)
            #     for item_name, paths in content_dict.items():
            #         markdown_content += f"### {item_name}\n"
            #         for i, path in enumerate(paths):
            #             markdown_content += f"![Option {i+1}]({path})\n"
            pass
        elif role == 'user':
            markdown_content += f"**User Query:** {content}\n\n"
            pass
        elif role == 'assistant':
            markdown_content += f"\n{content}\n\n"
    
    return markdown_content
markdown_content = generate_markdown_from_session(session_messages)
display(Markdown(markdown_content))


**User Query:** I am a girl going to a festival, could you please suggest an outfit I can wear!


Sure! Festivals are a great place to showcase fun, vibrant, and trendy outfits. I'll search for each item needed:

1. A stylish top
2. Comfortable yet fashionable bottom (shorts or skirt)
3. Suitable shoes for walking
4. Cool accessories like sunglasses or a hat

Let me query the database to find some great suggestions for you.


Here are some outfit suggestions for a festival. I've picked options that are colorful, trendy, and comfortable, perfect for a festival setting. 

### Tops
|  |  |  |  |  |
|:-:|:-:|:-:|:-:|:-:|
|![](../data/raw/images\13523.jpg)|![](../data/raw/images\52118.jpg)|![](../data/raw/images\57898.jpg)|![](../data/raw/images\42401.jpg)|![](../data/raw/images\42515.jpg)|

### Bottoms (Shorts/Skirts)
|  |  |  |  |  |
|:-:|:-:|:-:|:-:|:-:|
|![](../data/raw/images\33145.jpg)|![](../data/raw/images\38938.jpg)|![](../data/raw/images\38939.jpg)|![](../data/raw/images\29173.jpg)|![](../data/raw/images\38505.jpg)|

### Shoes
|  |  |  |  |  |
|:-:|:-:|:-:|:-:|:-:|
|![](../data/raw/images\17679.jpg)|![](../data/raw/images\14371.jpg)|![](../data/raw/images\14380.jpg)|![](../data/raw/images\2832.jpg)|![](../data/raw/images\34518.jpg)|

### Accessories (Sunglasses/Hat)
|  |  |  |  |  |
|:-:|:-:|:-:|:-:|:-:|
|![](../data/raw/images\33731.jpg)|![](../data/raw/images\51672.jpg)|![](../data/raw/images\33730.jpg)|![](../data/raw/images\51682.jpg)|![](../data/raw/images\23788.jpg)|

You can mix and match these to create a perfect festival look. Enjoy your time at the festival!

